#### Importing necessary Libraries

In [0]:
import requests
import uuid
from datetime import datetime
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType, ArrayType
from pyspark.sql.functions import from_unixtime

#### Creating Log Table

In [0]:
%sql 

CREATE TABLE IF NOT EXISTS weather_log_table ( 
      id STRING,
      load_type STRING,
      table_name STRING,
      process_start_time TIMESTAMP,
      process_end_time TIMESTAMP,
      status STRING,
      comments STRING,
      start_date_time TIMESTAMP,
      end_date_time TIMESTAMP,
      created_on TIMESTAMP,
      created_by STRING
    )
    
    

#### Decorator Function To Update Log Table

In [0]:
def keep_log(func):
    def wrapper(*args, **kwargs):
        
        import uuid
        from datetime import datetime
        from pyspark.sql.functions import col, udf
        from pyspark.sql.types import TimestampType

        id = str(uuid.uuid4())
        load_type = args[0]
        table_name = args[1]
        process_start_dt = datetime.now()
        name = 'Nandan Singh'
        status = 'EXTRACTING'

        query = f"insert into weather_log_table (id, load_type, table_name, process_start_time, status, created_on, created_by)\
            values ('{id}', '{load_type}', '{table_name}', '{process_start_dt}', '{status}', '{process_start_dt}', '{name}')"
        spark.sql(query)
        
        try:
            df, start, end = func(*args[2:], **kwargs)
            status = 'COMPLETED'
        except Exception as e:
            status = 'ERROR'
            process_end_dt = datetime.now()
            query = f"update weather_log_table \
                        set process_end_time = '{process_end_dt}', status='{status}', comments='{e}',\
                        where id='{id}'"
            spark.sql(query)
            dbutils.notebook.exit(1)
        
        
        
        udf_id = udf(lambda : id)
        udf_created_on = udf(lambda : process_start_dt, TimestampType())
        udf_created_by = udf(lambda : name)
        
        df = df.withColumn('load_run_id', udf_id())
        df = df.withColumn('created_on', udf_created_on())
        df = df.withColumn('created_by', udf_created_by())
        
        df.write.format('delta').mode('append').saveAsTable(table_name)
        
        process_end_dt = datetime.now()
        query = f"update weather_log_table \
                    set process_end_time = '{process_end_dt}', status='{status}', start_date_time='{start}',\
                        end_date_time='{end}'\
                    where id='{id}'"
        spark.sql(query)
        
        return df
    
    return wrapper